<a href="https://colab.research.google.com/github/maegnji/XGBoost/blob/main/221024_ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 경로 설정 (공백있을시에 앞에 \필요)
%cd /content/drive/MyDrive/python/XGBoost/Chapter02

/content/drive/MyDrive/python/XGBoost/Chapter02


In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [6]:
df_census = pd.read.csv('census_cleaned.csv')

AttributeError: ignored